# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The dataset collected is related to 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts. During these phone campaigns, an attractive long-term deposit application, with good interest rates, was offered. For each contact, a large number of attributes was stored (e.g. see Table 2) and if there was a success (the target variable). For the whole database considered, there were 6499 successes (8% success rate).

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [ ]:
import pandas as pd

In [ ]:
original_df = pd.read_csv('data/bank-additional-full.csv', sep=';')

In [ ]:
original_df.head()
display(original_df.head())
original_df.info()
original_df.describe()

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

The business objective of this task is to develop a predictive model that can effectively identify which bank clients are more likely to subscribe to a term deposit.

In [ ]:
original_df.info()

After reviewing the df, here are some notes,
1. There are no null values in the dataset 
2. There are some 'unknown' values in the provided dataset in some columns. 'default' column had ~20% unknown values which is notable
3. pdays column can be binary feature (previously_contacted yes/no, where yes means pdays != 999 and no means pdays == 999)
4. Duration column can be dropped per the data description


In [ ]:
# First lets drop the duration column
original_df.drop(columns=['duration'], inplace=True)

# lets encode pdays column where 999 is 0 and rest is 1
# original_df['pdays'].value_counts()
original_df['pdays'] = original_df['pdays'].apply(lambda x: 0 if x == 999 else 1)

# verify the changes by value_counts on all the columns above
# pdays_unique = original_df['pdays'].unique()
# month_unique = sorted(original_df['month'].unique())
# loan_unique = sorted(original_df['loan'].unique())
# housing_unique = sorted(original_df['housing'].unique())
# y_unique = sorted(original_df['y'].unique())
# poutcome_unique = sorted(original_df['poutcome'].unique())

# print(f"pdays_unique: {pdays_unique}")
# print(f"month_unique: {month_unique}")
# print(f"loan: {loan_unique}")
# print(f"housing: {housing_unique}")
# print(f"y: {y_unique}")
# print(f"poutcome_unique: {poutcome_unique}")

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

target = 'y'
X = original_df.drop(columns=['y'])

numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
print(f"Numerical features: {numerical_features}")
print(f"Categorical features: {categorical_features}")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ]
)
y = LabelEncoder().fit_transform(original_df[target])

print(f"X shape: {X.shape}")
print(f"X columns: {X.columns}")

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
from sklearn.model_selection import train_test_split

# Using stratify=y to maintain the same class distribution in both sets
# because the target is imbalanced (11% 'yes', 89% 'no')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X_train_processed shape: {X_train_processed.shape}")
print(f"X_test_processed shape: {X_test_processed.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
# Notes:
# target has 88% no and 12% yes 
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

dummy_classifier_pred = DummyClassifier(strategy='most_frequent').fit(X_train_processed, y_train).predict(X_test_processed)

# Evaluate
baseline_accuracy = accuracy_score(y_test, dummy_classifier_pred)
print(f"Baseline Accuracy: {baseline_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, dummy_classifier_pred, zero_division=0))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, dummy_classifier_pred))

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=10000)

log_reg.fit(X_train_processed, y_train)
y_pred_log_res = log_reg.predict(X_test_processed)

### Problem 9: Score the Model

What is the accuracy of your model?

In [ ]:
print("Logistic Regression Classifier Performance:\n")
print("Accuracy:", accuracy_score(y_test, y_pred_log_res))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log_res))
print("\nClassification Report:\n", classification_report(y_test, y_pred_log_res))

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import time

models = {
    'Logistic Regression': log_reg,
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC()
}

results = []

for name, model in models.items():
    start_time = time.time()
    model.fit(X_train_processed, y_train)
    train_time = time.time() - start_time
    train_accuracy = accuracy_score(y_train, model.predict(X_train_processed))
    test_accuracy = accuracy_score(y_test, model.predict(X_test_processed))
    results.append({
        'Model': name,
        'Train Time (sec)': round(train_time, 2),
        'Train Accuracy': round(train_accuracy, 2),
        'Test Accuracy': round(test_accuracy, 2)
    })

results_df = pd.DataFrame(results)
print("Model Comparison Results:")
results_df

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [ ]:
from sklearn.model_selection import GridSearchCV

model_param_dict = {
    'Logistic Regression': {
        'model': LogisticRegression(random_state=42, max_iter=2000),
        'params': {
            'C': [0.01, 0.1, 1, 10, 100],
            'solver': ['liblinear', 'saga'],
            'class_weight': ['balanced', None]
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9, 11],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'max_depth': [None, 5, 10, 15, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'class_weight': ['balanced', None]
        }
    },
    'SVM': {
        'model': SVC(random_state=42),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale']
        }
    }
}

def evaluate_models(model_param_dict, X_train, X_test, y_train, y_test):
    print(f"starting model evaluation")
    results = []
    best_models = {}
    best_params = {}
    
    for model_name, model_info in model_param_dict.items():
        print(f"evaluating {model_name}..")
        start_time = time.time()
        grid_search = GridSearchCV(
            estimator=model_info['model'],
            param_grid=model_info['params'],
            cv=5,
            scoring='recall',
            n_jobs=-1
        )
        grid_search.fit(X_train, y_train)
        train_time = time.time() - start_time
        print(f"train_time is {train_time}")
        
        best_models[model_name] = grid_search.best_estimator_
        best_params[model_name] = grid_search.best_params_
        
        y_train_pred = best_models[model_name].predict(X_train)
        y_test_pred = best_models[model_name].predict(X_test)

        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        
        results.append({
            'Model': model_name,
            'Train Time (sec)': round(train_time, 2),
            'Best Parameters': str(grid_search.best_params_),
            'Train Accuracy': round(train_accuracy, 2),
            'Test Accuracy': round(test_accuracy, 2),
            'CV Score': round(grid_search.best_score_, 2)
        })
        
        print(f"Completed {model_name} evaluation")
    
    results_df = pd.DataFrame(results)
    
    return {
        'best_models': best_models,
        'best_params': best_params,
        'results': results_df
    }

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
hyper_p_results_df = pd.DataFrame(evaluate_models(model_param_dict, X_train_processed, X_test_processed, y_train, y_test)['results'])
hyper_p_results_df

##### Questions